📁 Resources

In [23]:
import pandas as pd
import time
import random
import dataextension as dx
import pandas_flavor as pf
import requests
# from pandasgui import show
from config.settings import DATA_DIR
from config.settings import URLS_DIR
import emailaddress
from bs4 import BeautifulSoup


keywords = pd.read_csv(rf"{DATA_DIR}keywords.csv")["selection"].dropna().tolist()
bannedwords = pd.read_csv(rf"{DATA_DIR}bannedwords.csv")["job_title_de"].dropna().to_list()

📁 Load links

In [24]:

links  = pd.read_csv(rf"{URLS_DIR}stellenmarktdirektde_urls.csv")
df = links[["url","headers"]][:2]
df.to_csv(rf"{DATA_DIR}stellenmarktdirektde.csv",index=False)

📁 Generate initial content

In [25]:

def icontent(row):
    # time.sleep(random.randint(1,3))
    response = requests.get(row["url"],headers=eval(row["headers"]))
    bs = BeautifulSoup(response.content,"lxml")
    return bs.find_all("div",{"class":"item"})
 
    
df = pd.read_csv(rf"{DATA_DIR}stellenmarktdirektde.csv")
df = df.insertone(with_func=icontent,column_name="icontent", errors=True, description="Initial content")
df.to_csv(rf"{DATA_DIR}stellenmarktdirektde.csv",index=False)
# df.to_csv(rf"{DATA_DIR}stellenmarktdirektde_backup.csv",index=False)

Initial content: 100%|██████████| 2/2 [01:24<00:00, 42.06s/it]


📁 Convert icontent to list

In [26]:
import json
import ast
import re
def convert_to_list(x):
    print(type(x))
    if not dx.is_blank(x):
        return eval(x)
    return x

df = pd.read_csv(rf"{DATA_DIR}stellenmarktdirektde.csv")
df = df.astype('object')
df = df[df["icontent"].apply(lambda x: not dx.is_blank(x))]
df["icontent"] = df["icontent"].progress_apply(convert_to_list)
df = df.explode("icontent",ignore_index=True)
# df.to_csv(rf"{DATA_DIR}stellenmarktdirektde.csv",index=False)
df.info()
 

Initial content: 0it [00:00, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       0 non-null      object
 1   headers   0 non-null      object
 2   icontent  0 non-null      object
dtypes: object(3)
memory usage: 132.0+ bytes


📁 Extract Job Title de

In [ ]:

def job_title_de(row): 
    return eval(row["icontent"])["title"]

df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=job_title_de, description="job_title_de")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Select positive Job Title de

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.select_positive(words=keywords)
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Delete Negative Job Title de

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.delete_negative(words=bannedwords)
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Company Name "company_name"

In [ ]:
def company_name(row):
    return eval(row['icontent'])['employer']['name']

df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=company_name, description="company_name")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Drop duplicates base on "company_name" and "job_title"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df = df.drop_duplicates(subset=['company_name','job_title_de'],keep='first')
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Extract Job Title EN "job_title_en"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df['job_title_en'] = df['job_title_de']
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Source url "source_url"

In [ ]:
def source_url(row):
    jobid =  eval(row['icontent'])['id']
    return f"https://ec.europa.eu/eures/portal/jv-se/jv-details/{jobid}?jvDisplayLanguage=de&lang=en"
   
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=source_url, description="source_url")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get json Source url "jsource_url"

In [ ]:
def jsource_url(row):
    jobid =  eval(row['icontent'])['id']
    return f"https://ec.europa.eu/eures/eures-apps/searchengine/page/jv/id/{jobid}?lang=en"
   
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=jsource_url, description="jsource_url")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Job Description DE "job_description_de"

In [ ]:
def job_description_de(row): 
    return eval(row["icontent"])["description"]
    
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=job_description_de, description="job_description_de")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Job Description EN "job_description_en"

In [ ]:
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df['job_description_en'] = df['job_description_de']
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get employment_type "job_employment_type"

In [ ]:
def employment_type(row): 
    return eval(row['icontent'])['positionScheduleCodes'][0]
     
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=employment_type, description="employment_type")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Company Website "company_website"

In [ ]:
def company_website(row):
    return eval(row['icontent'])['employer']['website']
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=company_website, description="company_website")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Get second page json content "jcontent"

In [ ]:
def jcontent(row):
    time.sleep(random.randint(1,3))
    response = requests.get(row["jsource_url"])
    return str(response.json())
 
    
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=jcontent,description="jcontent")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.to_csv(rf"{DATA_DIR}europade_backup.csv",index=False)

📁 Get City "city"

In [ ]:
def city(row):
    return eval(row['jcontent'])['jvProfiles']['de']['locations'][0]['cityName']
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=city, description="city")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get Full Name "full_name"

In [ ]:
def full_name(row):
    first_name = eval(row['jcontent'])['jvProfiles']['de']['personContacts'][0].get('givenName','')
    last_name = eval(row['jcontent'])['jvProfiles']['de']['personContacts'][0].get('familyName','')
    return f"{first_name} {last_name}"
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=full_name, description="full_name")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)

📁 Get contact_email "contact_email"

In [ ]:
def contact_email(row):
    email = emailaddress.from_text(text=row['job_description_de'])
    if email:
        return email[0]
  
df = pd.read_csv(rf"{DATA_DIR}europade.csv")
df.inorup(with_func=contact_email, description="contact_email")
df.to_csv(rf"{DATA_DIR}europade.csv",index=False)
df.info()